In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import scipy.signal as scisig
import scipy.stats

#Sampling Frequencies
smp_frq_dt = {'ACC': 32, 'BVP': 64, 'EDA': 4, 'TEMP': 4, 'label': 700, 'Resp': 700}
WINDOW_IN_SECONDS = 30
label_dt = {'baseline': 1, 'stress': 2, 'amusement': 0}
int_label = {1: 'baseline', 2: 'stress', 0: 'amusement'}
feature = None
save_path = 'data'
sub_feature_path = '/subject_data'

if not os.path.exists(save_path):
    os.makedirs(save_path)
if not os.path.exists(save_path + sub_feature_path):
    os.makedirs(save_path + sub_feature_path)

# cvxEDA
def eda_stats(y):
    fq = smp_frq_dt['EDA']
    yn = (y - y.mean()) / y.std()
    [r, p, t, l, d, e, obj] = cvxEDA(yn, 1. / fq)
    return [r, p, t, l, d, e, obj]


class SubjectData:

    def __init__(self, main_path, subject_number):
        self.name = f'S{subject_number}'
        self.subject = ['signal', 'label', 'subject']
        self.signal = ['chest', 'wrist']
        self.chest = ['ACC', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp']
        self.wrist = ['ACC', 'BVP', 'EDA', 'TEMP']
        with open(os.path.join(main_path, self.name) + '/' + self.name + '.pkl', 'rb') as file:
            self.data = pickle.load(file, encoding='latin1')
        self.labels = self.data['label']

    def wrist_data(self):
        data = self.data['signal']['wrist']
        data.update({'Resp': self.data['signal']['chest']['Resp']})
        return data

    def chest_data(self):
        return self.data['signal']['chest']

    def extract_features(self):  # only wrist
        results = \
            {
                key: get_statistics(self.wrist_data()[key].flatten(), self.labels, key)
                for key in self.wrist_keys
            }
        return results


#Lowpass Filter
def lowpass(cutoff, fs, order=5):
    # Filtering Helper functions
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = scisig.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

#Application of Lowpass Filter
def lowpass_filter(data, cutoff, fs, order=5):
    # Filtering Helper functions
    b, a = lowpass(cutoff, fs, order=order)
    y = scisig.lfilter(b, a, data)
    return y
#Get Slope of Temperature
def get_slope(series):
    linear_reg = scipy.stats.linregress(np.arange(len(series)), series )
    slope = linear_reg[0]
    return slope
#Compute statistics the Features
def compute_window_stats(data, label=-1):
    mean_features = np.mean(data)
    std_features = np.std(data)
    min_features = np.amin(data)
    max_features = np.amax(data)

    features = {'mean': mean_features, 'std': std_features, 'min': min_features, 'max': max_features,
                'label': label}
    return features

#Three-axis Acceleration
def net_accel(data):
    return (data['ACC_x'] ** 2 + data['ACC_y'] ** 2 + data['ACC_z'] ** 2).apply(lambda x: np.sqrt(x))

#Get Peak Frequency for BVP
def peak_freq(x):
    frq, peak = scisig.periodogram(x, fs=8)
    psd_dict = {amp: freq for amp, freq in zip(peak, frq)}
    peak_freq = psd_dict[max(psd_dict.keys())]
    return peak_freq


# https://github.com/MITMediaLabAffectiveComputing/eda-explorer/blob/master/AccelerometerFeatureExtractionScript.py
#Filter Signal for Acceleration
def filterSignal(eda, cutoff=0.4, num=64):
    f = cutoff / (smp_frq_dt['ACC'] / 2.0)
    filter_coeff = scisig.firwin(num, f)

    return scisig.lfilter(filter_coeff, 1, eda)


def compute_features(e4_data_dict, labels, norm_type=None):
    # Dataframes for each sensor type
    eda_df = pd.DataFrame(e4_data_dict['EDA'], columns=['EDA'])
    bvp_df = pd.DataFrame(e4_data_dict['BVP'], columns=['BVP'])
    acc_df = pd.DataFrame(e4_data_dict['ACC'], columns=['ACC_x', 'ACC_y', 'ACC_z'])
    temp_df = pd.DataFrame(e4_data_dict['TEMP'], columns=['TEMP'])
    label_df = pd.DataFrame(labels, columns=['label'])
    resp_df = pd.DataFrame(e4_data_dict['Resp'], columns=['Resp'])

    # Filter EDA
    eda_df['EDA'] = lowpass_filter(eda_df['EDA'], 1.0, smp_frq_dt['EDA'], 6)

    # Filter ACM
    for _ in acc_df.columns:
        acc_df[_] = filterSignal(acc_df.values)

    # Adding indices for combination due to differing sampling frequencies
    eda_df.index = [(1 / smp_frq_dt['EDA']) * i for i in range(len(eda_df))]
    bvp_df.index = [(1 / smp_frq_dt['BVP']) * i for i in range(len(bvp_df))]
    acc_df.index = [(1 / smp_frq_dt['ACC']) * i for i in range(len(acc_df))]
    temp_df.index = [(1 / smp_frq_dt['TEMP']) * i for i in range(len(temp_df))]
    label_df.index = [(1 / smp_frq_dt['label']) * i for i in range(len(label_df))]
    resp_df.index = [(1 / smp_frq_dt['Resp']) * i for i in range(len(resp_df))]

    # Change indices to datetime
    eda_df.index = pd.to_datetime(eda_df.index, unit='s')
    bvp_df.index = pd.to_datetime(bvp_df.index, unit='s')
    temp_df.index = pd.to_datetime(temp_df.index, unit='s')
    acc_df.index = pd.to_datetime(acc_df.index, unit='s')
    label_df.index = pd.to_datetime(label_df.index, unit='s')
    resp_df.index = pd.to_datetime(resp_df.index, unit='s')

    # New EDA features
    r, p, t, l, d, e, obj = eda_stats(eda_df['EDA'])
    eda_df['EDA_phasic'] = r
    eda_df['EDA_smna'] = p
    eda_df['EDA_tonic'] = t
        
    # Combined dataframe which is not used
    df = eda_df.join(bvp_df, how='outer')
    df = df.join(temp_df, how='outer')
    df = df.join(acc_df, how='outer')
    df = df.join(resp_df, how='outer')
    df = df.join(label_df, how='outer')
    df['label'] = df['label'].fillna(method='bfill')
    df.reset_index(drop=True, inplace=True)

    if norm_type is 'std':
        # std norm
        df = (df - df.mean()) / df.std()
    elif norm_type is 'minmax':
        # minmax norm
        df = (df - df.min()) / (df.max() - df.min())

    # Groupby label
    grouped = df.groupby('label')
    baseline = grouped.get_group(1)
    stress = grouped.get_group(2)
    amusement = grouped.get_group(3)
    return grouped, baseline, stress, amusement

#Down sampling
def get_samples(data, n_windows, label):
    global feature
    global WINDOW_IN_SECONDS

    samples = []
    window_len = smp_frq_dt['label'] * WINDOW_IN_SECONDS

    for i in range(n_windows):
        # Get window of data
        w = data[window_len * i: window_len * (i + 1)]

        # Add acc        
        w = pd.concat([w, net_accel(w)])        
        cols = list(w.columns)
        cols[0] = 'net_acc'
        w.columns = cols
        
        # Calculate statistics for window
        wstats = compute_window_stats(data=w, label=label)

        # Seperating sample and label
        x = pd.DataFrame(wstats).drop('label', axis=0)
        y = x['label'][0]
        x.drop('label', axis=1, inplace=True)

        if feature is None:
            feature = []
            for row in x.index:
                for col in x.columns:
                    feature.append('_'.join([row, col]))

        # sample dataframe
        wdf = pd.DataFrame(x.values.flatten()).T
        wdf.columns = feature
        wdf = pd.concat([wdf, pd.DataFrame({'label': y}, index=[0])], axis=1)
        
        # Additional features
        wdf['BVP_peak_freq'] = peak_freq(w['BVP'].dropna())
        wdf['TEMP_slope'] = get_slope(w['TEMP'].dropna())
        samples.append(wdf)

    return pd.concat(samples)


def make_patient_data(subject_id):
    global save_path
    global WINDOW_IN_SECONDS

    # Make subject data object for Sn
    # Add your path to Dataset
    subject = SubjectData(main_path='Path to Dataset', subject_number=subject_id)

    # Empatica E4 data with RESP
    e4_data_dict = subject.wrist_data()

    norm_type = None

    # Classifying Three Classes
    grouped, baseline, stress, amusement = compute_features(e4_data_dict, subject.labels, norm_type)    
    n_baseline_wdws = int(len(baseline) / (smp_frq_dt['label'] * WINDOW_IN_SECONDS))
    n_stress_wdws = int(len(stress) / (smp_frq_dt['label'] * WINDOW_IN_SECONDS))
    n_amusement_wdws = int(len(amusement) / (smp_frq_dt['label'] * WINDOW_IN_SECONDS))  
    
    # Downsampling
    baseline_samples = get_samples(baseline, n_baseline_wdws, 1)
    # baseline_samples = baseline_samples[::2]
    stress_samples = get_samples(stress, n_stress_wdws, 2)
    amusement_samples = get_samples(amusement, n_amusement_wdws, 0)

    all_samples = pd.concat([baseline_samples, stress_samples, amusement_samples])
    all_samples = pd.concat([all_samples.drop('label', axis=1), pd.get_dummies(all_samples['label'])], axis=1)    
    all_samples.to_csv(f'{save_path}{sub_feature_path}/S{subject_id}_data.csv')    
    subject = None

# Combine Dataframes and Make CSV
def combine_files(subjects):
    df_list = []
    for s in subjects:
        df = pd.read_csv(f'{save_path}{sub_feature_path}/S{s}_data.csv', index_col=0)
        df['subject'] = s
        df_list.append(df)

    df = pd.concat(df_list)

    df['label'] = (df['0'].astype(str) + df['1'].astype(str) + df['2'].astype(str)).apply(lambda x: x.index('1'))
    df.drop(['0', '1', '2'], axis=1, inplace=True)

    df.reset_index(drop=True, inplace=True)
    # Make csv
    df.to_csv(f'{save_path}/stress.csv')

    counts = df['label'].value_counts()
    print('Number of samples per class:')
    for label, number in zip(counts.index, counts.values):
        print(f'{int_label[label]}: {number}')


if __name__ == '__main__':

    subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]

    for patient in subject_ids:
        print(f'Processing data for S{patient}...')
        make_patient_data(patient)
    #For combining Data
    combine_files(subject_ids)
    print('Processing complete.')